In [1]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 785.1/785.1 KB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 752.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 7

In [2]:
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt

--2023-02-15 05:58:07--  https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6208417 (5.9M) [text/plain]
Saving to: ‘dbpedia_sample_abstract_20k_unprep.txt.1’

dbpedia_sample_abst 100%[===================>]   5.92M  --.-KB/s    in 0.03s   

2023-02-15 05:58:07 (191 MB/s) - ‘dbpedia_sample_abstract_20k_unprep.txt.1’ saved [6208417/6208417]



In [3]:
!head -n 1 dbpedia_sample_abstract_20k_unprep.txt

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry


In [4]:
text_file = "dbpedia_sample_abstract_20k_unprep.txt"

In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [6]:
nltk.download('stopwords')

documents = [line.strip() for line in open(text_file, encoding='utf-8').readlines()]
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
preprocessed_documents[:2]

['mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry',
 'died march american photographer specialized photography operated studio silver spring maryland later lived florida magazine photographer year']

In [8]:
unpreprocessed_corpus[:2]

['The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry',
 "Monte Zucker (died March 15, 2007) was an American photographer. He specialized in wedding photography, entering it as a profession in 1947. In the 1970s he operated a studio in Silver Spring, Maryland. Later he lived in Florida. He was Brides Magazine's Wedding Photographer of the Year for 1990 and"]

In [9]:
print(len(vocab))

2000


In [10]:
vocab[:5]

['manga', 'considered', 'top', 'died', 'occurred']

In [11]:
tp = TopicModelDataPreparation('paraphrase-distilroberta-base-v1')

training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow = preprocessed_documents)

Batches:   0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
tp.vocab[:10]

['abbreviated',
 'academic',
 'academy',
 'access',
 'according',
 'achieved',
 'acquired',
 'acre',
 'acres',
 'across']

In [13]:
len(tp.vocab)

2000

In [15]:
set(vocab) == set(tp.vocab)

True

In [16]:
# Combined TM 학습
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [400000/400000]	Train Loss: 135.42655336914063	Time: 0:00:07.032629: : 20it [02:02,  6.11s/it]


In [18]:
# 토픽 확인
ctm.get_topic_lists(5)

[['station', 'line', 'railway', 'rail', 'metro'],
 ['state', 'election', 'held', 'members', 'united'],
 ['study', 'religious', 'form', 'term', 'philosophy'],
 ['island', 'river', 'islands', 'coast', 'part'],
 ['church', 'de', 'roman', 'diocese', 'century'],
 ['published', 'book', 'century', 'greek', 'written'],
 ['university', 'year', 'school', 'division', 'team'],
 ['team', 'season', 'football', 'played', 'games'],
 ['born', 'professor', 'law', 'university', 'studied'],
 ['found', 'mm', 'family', 'moth', 'brown'],
 ['kilometres', 'region', 'mi', 'area', 'municipality'],
 ['series', 'television', 'show', 'aired', 'premiered'],
 ['american', 'new', 'born', 'york', 'university'],
 ['club', 'league', 'team', 'player', 'professional'],
 ['league', 'club', 'football', 'team', 'played'],
 ['world', 'held', 'place', 'event', 'took'],
 ['united', 'states', 'organization', 'national', 'non'],
 ['county', 'city', 'states', 'united', 'located'],
 ['built', 'house', 'building', 'story', 'street'],

In [19]:
# Topic visualize
lda_vis_data = ctm.get_ldavis_data_format(tp.vocab, training_dataset, n_samples=10)

Sampling: [10/10]: : 10it [00:50,  5.01s/it]


In [20]:
import pyLDAvis as vis

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [21]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5)

Sampling: [5/5]: : 5it [00:26,  5.29s/it]


In [22]:
print(preprocessed_documents[0])

mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry


In [23]:
import numpy as np
topic_number = np.argmax(topics_predictions[0])

In [25]:
ctm.get_topic_lists(5)[topic_number]

['state', 'route', 'park', 'river', 'states']

In [26]:
# 모델 저장
ctm.save(models_dir='./')

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:470: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [27]:
del ctm

In [30]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)
ctm.load('/content/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99', epoch=19)

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:495: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [31]:
ctm.get_topic_lists(5)

[['station', 'line', 'railway', 'rail', 'metro'],
 ['state', 'election', 'held', 'members', 'united'],
 ['study', 'religious', 'form', 'term', 'philosophy'],
 ['island', 'river', 'islands', 'coast', 'part'],
 ['church', 'de', 'roman', 'diocese', 'century'],
 ['published', 'book', 'century', 'greek', 'written'],
 ['university', 'year', 'school', 'division', 'team'],
 ['team', 'season', 'football', 'played', 'games'],
 ['born', 'professor', 'law', 'university', 'studied'],
 ['found', 'mm', 'family', 'moth', 'brown'],
 ['kilometres', 'region', 'mi', 'area', 'municipality'],
 ['series', 'television', 'show', 'aired', 'premiered'],
 ['american', 'new', 'born', 'york', 'university'],
 ['club', 'league', 'team', 'player', 'professional'],
 ['league', 'club', 'football', 'team', 'played'],
 ['world', 'held', 'place', 'event', 'took'],
 ['united', 'states', 'organization', 'national', 'non'],
 ['county', 'city', 'states', 'united', 'located'],
 ['built', 'house', 'building', 'story', 'street'],